# PCA-PCA on cora Dataset

# Setting/Initialization

In [34]:
import os
import shutil
import json
import networkx as nx
import numpy as np
from collections import OrderedDict
import functools as ft
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, rand_score
from sklearn import model_selection as sk_ms
from sklearn.svm import LinearSVC
from sklearn import preprocessing
import functools as ft
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, rand_score, davies_bouldin_score, calinski_harabasz_score
from nltk.cluster.util import cosine_distance, euclidean_distance
from nltk.cluster.kmeans import KMeansClusterer
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import networkx as nx
from sklearn.svm import SVC  # or another classifier of your choice
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class Util_class():

  """
  dict_in : dictionary
  list_key : list of key
  return True if ALL key in list_key is in dict
  """
  def check_key_in_dict(dict_in, list_key):
    is_in = True
    list_not_in = []
    for key in list_key:
      if key not in dict_in:
        is_in = False
        list_not_in.append(key)
    return [is_in,list_not_in]

  def same_key_in_dict(dict_in, list_key):
    is_in = True
    key_not_dict = []
    key_not_list = []

    for key in list_key:
      if key not in dict_in:
        is_in = False
        key_not_dict.append(key)

    for key in dict_in:
      if key not in list_key:
        is_in = False
        key_not_list.append(key)
    return [is_in,key_not_dict,key_not_list]

  def folder_manage(path, uniquify=True,clean=False, force=False):
    last_folder = os.path.basename(os.path.normpath(path))
    head_path = os.path.dirname(os.path.normpath(path))

    #head of path exist
    if os.path.exists(head_path):
        #path last folder not exist
        if not os.path.exists(path):
            os.makedirs(path)
            return os.path.normpath(path)
        #path last folder  exist
        else:
            if uniquify:
                counter = 1
                while os.path.exists(path):
                    path = head_path + "/" + last_folder  + "(" + str(counter) + ")"
                    counter += 1
                os.makedirs(path)
            #empty last folder
            elif clean:
                if force:
                    shutil.rmtree(path)
                    os.makedirs(path)
                else:
                    print(f'Enter YES or Y to delete all file or directory from: {path}')
                    input_clean = input()
                    if input_clean in ["YES","Y","yes","y"]:
                        shutil.rmtree(path)
                        os.makedirs(path)
                    else:
                        raise Util_class_folder_manage_forceDelete(path)
        return os.path.normpath(path)
    else:
        print(f'Enter YES or Y to create directories: {path}')
        input_clean = input()
        if input_clean in ["YES","Y","yes","y"]:
            os.makedirs(path)
            return os.path.normpath(path)
        else:
            raise Util_class_folder_manage_dirnameNotExist(head_path)

class Util_class_folder_manage_dirnameNotExist(Exception):
    """Exception raised for errors in activation function type"""

    def __init__(self, value):
        self.value = value

    def __str__(self):
        return f"Directory name '{self.value}' not exist."

class Util_class_folder_manage_forceDelete(Exception):
    """Exception raised for errors in activation function type"""

    def __init__(self, value):
        self.value = value

    def __str__(self):
        return f"Not possible force clean the folder: '{self.value}'."


datase_name = 'cora'
path_models_checkpoint = '/content/models_checkpoint'
Util_class.folder_manage(path_models_checkpoint)
path = '/content/dataset/'
Util_class.folder_manage(path)
!git clone https://github.com/MIND-Lab/CAGE.git


fatal: destination path 'CAGE' already exists and is not an empty directory.


In [35]:
import pandas as pd
import matplotlib.pyplot as plt


class PerformanceEmbedding():

    def __init__(self, model, labels, embedding_name='att', node_label='node_label'):
        if not isinstance(model, GraphEModel):
            raise PerformanceEmbedding_notModelClass(model)
        self.embedding = model.get_embedding(phase=embedding_name, type_output="numpy")
        self.labels = labe
        self.group_by = ['avg','sum']
        self.cluster_measure = ['rand_score']
        self.classifier_measure = ['accuracy_score','precision_macro','precision_micro','precision_weighted',
                                   'recall_macro','recall_micro','recall_weighted',
                                   'f1_macro','f1_micro','f1_weighted']

    def visualization(self):
        visualemb = VisualEmbedding(self.embedding,self.labels)
        return visualemb.embedding_visualization(None)

    def classification(self, repetitions = 10, classifier_name = "svm", performance_group_by='avg',labeled_data_threshold=None, measures_selected = None, random_set = True):
        if measures_selected is None:
            measures_selected = self.classifier_measure
        else:
            for meas in measures_selected:
                if meas not in self.classifier_measure:
                    raise PerformanceEmbedding_notMeasureExperiment(meas,'NodeClassification')
        if labeled_data_threshold is None:
            labeled_data_threshold = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


        n_classify = NodeClassification(self.embedding,self.labels, normalize=True)
        measures = dict()
        for split_threshold in labeled_data_threshold:
            measure = n_classify.classification(classifier_name, split_threshold, repetitions, self.group_by, random_set)
            _key = "split_"+str(split_threshold)
            measures[_key] = measure

        return self.performance_measure(measures,measures_selected,performance_group_by)

    def clusterization(self, repetitions = 10, performance_group_by='avg', measures_selected = None):
        if measures_selected is None:
            measures_selected = self.cluster_measure
        else:
            for meas in measures_selected:
                if meas not in self.cluster_measure:
                    raise PerformanceEmbedding_notMeasureExperiment(meas,'NodeClustering')

        n_clusterfy = NodeClustering(self.embedding,self.labels)
        measures = dict()
        measure = n_clusterfy.classic_clusterizzation(repetitions, self.group_by)
        measures["all"] = measure
        return self.performance_measure(measures,measures_selected,performance_group_by)


    def performance_measure(self, measures, measures_selected, groub_by='avg',):
        pd_measure = pd.DataFrame()
        pd_measure['name_measure'] = measures_selected

        for split_name in measures:
            val_col = list()
            for meas_name in measures_selected:
                if meas_name not in measures[split_name][groub_by]:
                    raise PerformanceEmbedding_notMeasure(meas_name)
                else:
                    val_col.append(measures[split_name][groub_by][meas_name])
            pd_measure[split_name] = val_col
        pd_measure.set_index('name_measure')
        return pd_measure


    def loss_plot(self):
        data_plot_losses = [val.item() for val in DAGE_values['losses']]
        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.plot(data_plot_losses,"b.")



class PerformanceEmbedding_notModelClass(Exception):
      """Exception raised for not classifier type found"""

      def __init__(self, obj):
          self.obj = obj

      def __str__(self):
          return f"Model should be a 'GraphEModel' class object but receive a ''{type(self.obj)} object."

class PerformanceEmbedding_notMeasure(Exception):
      """Exception raised for not classifier type found"""

      def __init__(self, name_measure):
          self.name_measure = name_measure

      def __str__(self):
          return f"Percormance '{self.name_measure}' not recognized."

class PerformanceEmbedding_notMeasureExperiment(Exception):
      """Exception raised for not classifier type found"""

      def __init__(self, measure_name, experiment_name):
          self.measure_name = measure_name
          self.experiment_name = experiment_name

      def __str__(self):
          return f"Experiment '{self.experiment_name}' not implement performance called '{self.measure_name}'."


In [36]:
class LoadDataset():

    def __init__(self,edge_file_name,attribute_file_name,label_file_name,attribute_file_format="normalized_matrix",is_directed_graph=False):
        """
        edge_file : file with all edge (pair of nodes)
        attribute_file : file with all attribute
        label_file :
        attribute_file_format : format od attribute data:
              "normal_matrix" : each row is alredy a frequency normalizzated vector (DEFAULT) es: CORA dataset
              "naive_text" : each row is item text description
        is_directed_graph : boolean, if true is a direct graph else (DEFAULT) is a undirect graph

        """
        self.is_directed_graph = is_directed_graph
        #input shape
        self.input_shape = dict()

        #Structural preprocessing
        self.edge_file_name = edge_file_name
        self.graph = self.edge_createGraph()
        #Changed due to nx not containing function
        self.edge_adj_matrix = np.array(nx.to_numpy_array(self.graph, nodelist=sorted(self.graph.nodes())))

        #Attribute preprocessing
        self.attribute_file_name = attribute_file_name
        self.attribute_adj_matrix = np.array(self.attribute_createMatrix(attribute_file_format))

        #Class preprocessing
        self.label_file_name = label_file_name
        self.label_vec = self.labels_createVector()




    def get_structural_matrix(self):
        return self.edge_adj_matrix

    def get_attribute_matrix(self):
        return self.attribute_adj_matrix

    def get_vector_matrix(self):
        return self.label_vec

    def get_graph(self):
        return self.graph

    def export_graph(self, pathfile, filename, extention="graphml"):
        path = pathfile+'/'+filename+'.'+extention

        if extention == "graphml":
            nx.write_graphml( self.graph, path)
        elif extention == "gml":
            nx.write_gml( self.graph, path)
        else:
            raise LoadDataset_Exception_Graph_FormatExport_notRecognized(extention)
        return True

    def get_input_shape(self, key):
        return self.input_shape[key]

    def edge_createGraph(self):
        if self.is_directed_graph:
            g = nx.DiGraph()
        else:
            g = nx.Graph()
        try:
            with open(self.edge_file_name, 'r') as edge_file:
                for line in edge_file:
                    edge = line.split()
                    if len(edge) == 3:
                        edge_weight = float(edge[2])
                    else:
                        edge_weight = 1.0
                    if len(edge) == 1:
                        g.add_node(int(edge[0]))
                    else:
                        g.add_edge(int(edge[0]), int(edge[1]), weight = edge_weight)
        except Exception as e:
            raise e
        self.input_shape['net'] = g.number_of_nodes()
        print("Structure dimension:\t",self.input_shape['net'])
        return g

    def attribute_createMatrix(self, attribute_file_format):
        if attribute_file_format == "normalized_matrix":
            try:
                att_matrix = []
                with open(self.attribute_file_name, 'r') as att_file:
                    for line in att_file:
                      att_line = line.replace("\n", "").split(" ")[1:]
                      att_matrix.append([float(n) for n in att_line])
                self.input_shape['att'] = len(att_matrix[0])
                print("Attribute dimension:\t",self.input_shape['att'])
                return att_matrix
            except Exception as e:
                raise e
        elif attribute_file_format == "naive_text":
            print("naive_text to do")
            try:
                att_matrix = []
                with open(self.attribute_file_name, 'r') as att_file:
                    for line in att_file:
                        print(line)
                        break
                    corpus = json.load(att_file)
                    print(corpus)

                return 0
            except Exception as e:
                raise e
        else:
            raise(LoadDataset_Exception_Attribute_Format_notRecognized(attribute_file_format))


    def labels_createVector(self):
        try:
            with open(self.label_file_name, 'r') as label_file:
                node_label_dict = {}
                for line in label_file:
                    split_line = line.replace("\n", "").split(" ")
                    node_id = int(split_line[0])
                    node_label = int(split_line[1])
                    node_label_dict[node_id] = node_label
                # sort the keys (node_ids) of the dictionary
                node_label_dict = OrderedDict(sorted(node_label_dict.items(), key=lambda t: t[0]))
                labels = np.array(list(node_label_dict.values()))
                return labels
        except Exception as e:
            raise e

    def get_labels(self):
        return self.label_vec


class LoadDataset_Exception_Attribute_Format_notRecognized(Exception):
      """Exception raised for errors in list of layers type"""

      def __init__(self, value):
          self.value = value

      def __str__(self):
          return f'{self.value} : type of attribute file format not recognized.'

class LoadDataset_Exception_Graph_FormatExport_notRecognized(Exception):
      """Exception raised for errors in list of layers type"""

      def __init__(self, value):
          self.value = value

      def __str__(self):
          return f'{self.value} : graph format export not recognized.'


# Testing Functions

In [37]:
class NodeClassification():

    def __init__(self, embedding_data, labels, normalize=False):
        if normalize:
            self.data = pd.DataFrame(preprocessing.normalize(embedding_data, norm='l2', axis=1))
        else:
            self.data = pd.DataFrame(embedding_data)
        self.oper_math = ['sum', 'avg']
        self.labels = labels
        self.performal_measure = {
            "accuracy_score" : ft.partial(accuracy_score),
            "precision_macro" : ft.partial(precision_score, average='macro'),
            "precision_micro" : ft.partial(precision_score, average='micro'),
            "recall_macro" : ft.partial(recall_score, average='macro'),
            "recall_micro" : ft.partial(recall_score, average='micro'),
            "f1_macro" : ft.partial(f1_score, average='macro', labels=np.unique(self.labels)),
            "f1_micro" : ft.partial(f1_score, average='micro', labels=np.unique(self.labels)),

            "precision_weighted" : ft.partial(precision_score, average='weighted'),
            "recall_weighted" : ft.partial(recall_score, average='weighted'),
            "f1_weighted" : ft.partial(f1_score, average='weighted', labels=np.unique(self.labels)),
        }

    def split_dataset(self, split_threshold, num_split, random_set):
        """
        split_threshold : float, threshold of test split
        num_split : int, number of split
        random_set : boolean, TRUE if each split is randomly different by others
        return : array, where each item is a dictionary of data's split, with keys X_train, X_test, Y_train, Y_test
        """
        data_splitted = list()
        _splitted_data = None
        for i in range(num_split):
            if random_set or (_splitted_data == None):
                X_train, X_test, Y_train, Y_test = sk_ms.train_test_split(self.data, self.labels, test_size=split_threshold)
                _splitted_data = {
                    "X_train" : X_train,
                    "Y_train" : Y_train,
                    "X_test" : X_test,
                    "Y_test" : Y_test,
                }
            data_splitted.append(_splitted_data)
        return data_splitted



    def classification(self, classifier_name, split_threshold, repetitions, group_by, random_set):
        """
        classifier_name : string, name of classifier. Implemented "svm",
        split_threshold : float, threshold of test split
        repetitions : int, number of repetitions of classification
        random_set : boolean, TRUE if each split is randomly different by others
        group_by : array, math operation group by measure
        return : array, each item is a data's split
        """
        data_splitted = self.split_dataset(split_threshold, repetitions, random_set)

        if classifier_name == "svm":
            classifier = LinearSVC()
        else:
            raise NodeClassification_notClassifierFound(classifier_name)
        measures_performance = dict()

        for n_repetition in range(repetitions):
            data_split = data_splitted.pop(0)
            classifier.fit(data_split["X_train"], data_split["Y_train"])
            predictions = self.prediction(classifier, data_split["X_test"])
            performance_computation = self.performance_computation(data_split["Y_test"], predictions)

            key_iter = "iter_"+str(n_repetition)
            measures_performance[key_iter] = performance_computation

        measure_total = dict()
        for oper in group_by:
            if oper in self.oper_math:
                measure_total[oper] = dict()
                for measure_name in self.performal_measure:
                      measure_total[oper][measure_name] = list()
            else:
                raise NodeClassification_notAggregationRecognizer(oper)

        for iteration_measure in measures_performance:
            for measure_name in self.performal_measure:
                value = measures_performance[iteration_measure][measure_name]
                for oper in group_by:
                    measure_total[oper][measure_name].append(value)
        for oper in group_by:
            for measure_name in self.performal_measure:
                if oper == "avg":
                    measure_total[oper][measure_name] = sum(measure_total[oper][measure_name])/len(measure_total[oper][measure_name])
                elif oper == "sum":
                    measure_total[oper][measure_name] = sum(measure_total[oper][measure_name])
                else:
                    measure_total[oper][measure_name] = 0
        return measure_total


    def prediction(self, model, data):
        return model.predict(data)

    def performance_computation(self, Y_test, Y_pred):
        performance_measure_computed = dict()

        for measure_name in self.performal_measure:
            measure_function = self.performal_measure[measure_name]
            measure_value = measure_function(Y_test, Y_pred)
            performance_measure_computed[measure_name] = measure_value
        return performance_measure_computed


class NodeClassification_notClassifierFound(Exception):
      """Exception raised for not classifier type found"""

      def __init__(self, name):
          self.name = name

      def __str__(self):
          return f"{type(self.name)} : type of classifier not found."

class NodeClassification_notAggregationRecognizer(Exception):
      """Exception raised for not classifier type found"""

      def __init__(self, name):
          self.name = name

      def __str__(self):
          return f" Classification not support '{type(self.name)}' group_by. It's accept: 'sum' or 'avg'."


In [41]:
edge_file_name = f"/Users/darkr/OneDrive/Documents/GitHub/CAGE/data/{datase_name}/in_edges.txt"
attribute_file_name = f"/Users/darkr/OneDrive/Documents/GitHub/CAGE/data/{datase_name}/in_features.txt"
label_file_name = f"/Users/darkr/OneDrive/Documents/GitHub/CAGE/data/{datase_name}/in_group.txt"

path_model_checkpoint = f"/content/models_checkpoint/CAGE_{datase_name}"
Util_class.folder_manage(path_model_checkpoint)

is_directed_graph = False
attribute_file_format = "normalized_matrix"
dataLoad = LoadDataset(edge_file_name,attribute_file_name,label_file_name,attribute_file_format=attribute_file_format,is_directed_graph=is_directed_graph)

dataLoad.export_graph(f"/content/models_checkpoint/CAGE_{datase_name}",f"CAGE_{datase_name}_graph")
net_adj = dataLoad.get_structural_matrix()
att_adj = dataLoad.get_attribute_matrix()
label_vec = dataLoad.get_labels()
print(datase_name)


Structure dimension:	 2708
Attribute dimension:	 1433
cora


# PCA Model

In [53]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC  # or another classifier of your choice

def read_edges(file_path):
    # Read edges from a text file and return a list of tuples (edges)
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            node1, node2 = line.strip().split()
            edges.append((int(node1), int(node2)))  # Convert to integers
    return edges

def read_features(file_path):
    # Read features from a text file and return a numpy array
    return np.loadtxt(file_path, delimiter=' ')

def create_adjacency_matrix(edges, num_nodes):
    # Create an adjacency matrix from a list of edges
    adjacency_matrix = np.zeros((num_nodes, num_nodes))
    for node1, node2 in edges:
        adjacency_matrix[int(node1), int(node2)] = 1
        adjacency_matrix[int(node2), int(node1)] = 1  # Assuming undirected graph
    return adjacency_matrix

def apply_pca(data, n_components=2):
    # Apply PCA to the given data
    pca = PCA(n_components=n_components)
    return pca.fit_transform(data)

def read_labels(label_file_path):
    """
    Read node labels/groups from a text file and return a 1D numpy array.
    Assumes the first column is the node number and the second column is the label.
    """
    all_data = np.loadtxt(label_file_path, dtype=int)
    labels = all_data[:, 1]  # Extract only the second column for labels
    return labels

def main(edge_file_name, attribute_file_name, label_file_name):
    # Read data
    edges = read_edges(edge_file_name)
    features = read_features(attribute_file_name)
    labels = read_labels(label_file_name)
    unique_classes = np.unique(labels)
    print("Unique classes in labels:", unique_classes)

    # Apply PCA to the features
    pca_features = apply_pca(features)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(pca_features, labels, test_size=0.2, random_state=42, stratify=labels)

    # Train a classifier
    classifier = SVC()  # Support Vector Classifier
    classifier.fit(X_train, y_train)

    # Predict and calculate accuracy
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Classification Accuracy:", accuracy)

    return pca_features, labels

if __name__ == "__main__":
    edge_file_name = f"/Users/darkr/OneDrive/Documents/GitHub/CAGE/data/{datase_name}/in_edges.txt"
    attribute_file_name = f"/Users/darkr/OneDrive/Documents/GitHub/CAGE/data/{datase_name}/in_features.txt"
    label_file_name = f"/Users/darkr/OneDrive/Documents/GitHub/CAGE/data/{datase_name}/in_group.txt"
    pca_embedding, labels = main(edge_file_name, attribute_file_name, label_file_name)


Unique classes in labels: [0 1 2 3 4 5 6]
Classification Accuracy: 0.3025830258302583


# Testing PCA

In [57]:
dataset_name = "cora"  # Replace with your dataset name

# Read embeddings and labels from the main function
pca_embeddings, labels = main(edge_file_name, attribute_file_name, label_file_name)

print("PCA Embeddings Shape:", pca_embeddings.shape)
print("Labels Shape:", labels.shape)

node_classifier = NodeClassification(pca_embeddings, labels, normalize=True)
evaluation_results = node_classifier.classification(classifier_name="svm", split_threshold=0.2, repetitions=10, group_by=["avg"], random_set=True)

print(evaluation_results)


Unique classes in labels: [0 1 2 3 4 5 6]
Classification Accuracy: 0.3025830258302583
PCA Embeddings Shape: (2708, 2)
Labels Shape: (2708,)


c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-package

{'avg': {'accuracy_score': 0.2968634686346864, 'precision_macro': 0.04240906694781233, 'precision_micro': 0.2968634686346864, 'recall_macro': 0.14285714285714282, 'recall_micro': 0.2968634686346864, 'f1_macro': 0.06535461254973766, 'f1_micro': 0.2968634686346864, 'precision_weighted': 0.08848974006345231, 'recall_weighted': 0.2968634686346864, 'f1_weighted': 0.13624464942120898}}


c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-package

# PCA PCA model

In [44]:
import numpy as np
from sklearn.decomposition import PCA
import networkx as nx
from sklearn.svm import SVC  # or another classifier of your choice
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def read_edges(file_path):
    # Read edges from a text file and return a list of tuples (edges)
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            node1, node2 = line.strip().split()
            edges.append((int(node1), int(node2)))  # Convert to integers
    return edges

def read_features(file_path):
    # Read features from a text file and return a numpy array
    return np.loadtxt(file_path, delimiter=' ')

def create_adjacency_matrix(edges, num_nodes):
    # Create an adjacency matrix from a list of edges
    adjacency_matrix = np.zeros((num_nodes, num_nodes))
    for node1, node2 in edges:
        adjacency_matrix[int(node1), int(node2)] = 1
        adjacency_matrix[int(node2), int(node1)] = 1  # Assuming undirected graph
    return adjacency_matrix

def apply_pca(data, n_components=2):
    # Apply PCA to the given data
    pca = PCA(n_components=n_components)
    return pca.fit_transform(data)

def read_labels(file_path):
    """
    Read node labels/groups from a text file and return a 1D numpy array.
    Assumes the first column is the node number and the second column is the label.
    """
    all_data = np.loadtxt(file_path, dtype=int)
    labels = all_data[:, 1]  # Extract only the second column for labels
    return labels

def merge_embeddings(structural_embedding, attribute_embedding):
    """
    Merge structural and attribute embeddings by concatenation.
    """
    return np.concatenate((structural_embedding, attribute_embedding), axis=1)

def main(dataset_name):
    # Read data
    edges = read_edges(edge_file_name)
    features = read_features(attribute_file_name)
    labels = read_labels(label_file_name)
    unique_classes = np.unique(labels)
    print("Unique classes in labels:", unique_classes)

    # Create a graph from edges
    G = nx.Graph()
    G.add_edges_from(edges)

    # Get number of nodes
    num_nodes = max(max(edge) for edge in edges) + 1

    # Create adjacency matrix
    adjacency_matrix = create_adjacency_matrix(edges, num_nodes)

    # Apply PCA for Structural Embedding
    structural_embedding = apply_pca(adjacency_matrix)

    # Apply PCA for Attribute Embedding
    attribute_embedding = apply_pca(features)

    # Merge Structural and Attribute Embeddings
    merged_embedding = merge_embeddings(structural_embedding, attribute_embedding)

    # Node Classification
    X_train, X_test, y_train, y_test = train_test_split(merged_embedding, labels, test_size=0.2, random_state=42, stratify=labels)

    classifier = SVC()  # Support Vector Classifier
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Classification Accuracy:", accuracy)

    return(merged_embedding)

if __name__ == "__main__":
    dataset_name = "cora"  # Replace with your dataset name
    main(dataset_name)


Unique classes in labels: [0 1 2 3 4 5 6]
Classification Accuracy: 0.3025830258302583


# Testing

In [ ]:
dataset_name = "cora"  # Replace with your dataset name

# Read embeddings and labels
pca_pca_embedding = main(dataset_name)
labels = read_labels(label_file_name)

node_classifier = NodeClassification(pca_pca_embedding, labels, normalize=True)
evaluation_results = node_classifier.classification(classifier_name="svm", split_threshold=0.2, repetitions=10, group_by=["avg"], random_set=True)

print(evaluation_results)


Unique classes in labels: [0 1 2 3 4 5 6]
Classification Accuracy: 0.3025830258302583


c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-package

{'avg': {'accuracy_score': 0.285239852398524, 'precision_macro': 0.04074855034264628, 'precision_micro': 0.285239852398524, 'recall_macro': 0.14285714285714282, 'recall_micro': 0.285239852398524, 'f1_macro': 0.06337637029030267, 'f1_micro': 0.285239852398524, 'precision_weighted': 0.08161245081085497, 'recall_weighted': 0.285239852398524, 'f1_weighted': 0.1268451127649292}}


c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\darkr\AppData\Local\Programs\PythonCodingPack\lib\site-package